## MLE Estimation with PyTorch

In [27]:
import numpy as np
import pandas as pd
import torch
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import warnings 
warnings.filterwarnings('ignore')
# -----------------------------------------------------------------
# 1. DATA SIMULATION: y = Xb + e, e ~ N(0, sigma^2)
# -----------------------------------------------------------------
np.random.seed(123)
n_obs = 1000
X_np = np.random.rand(n_obs, 2)  # 2 predictors, no intercept
beta_true = np.array([1.0, 2.0])
sigma_true = 1.5
y_np = X_np @ beta_true + np.random.normal(0, sigma_true, n_obs)

# -----------------------------------------------------------------
# 2. ANALYTICAL (CLOSED-FORM) MLE
#    beta_hat = (X'X)^(-1) X'y
#    sigma_hat^2 = (1/n)*Sum(e^2), e = y - Xb
# -----------------------------------------------------------------
XX_inv = np.linalg.inv(X_np.T @ X_np)
beta_analytic = XX_inv @ X_np.T @ y_np
resid_analytic = y_np - X_np @ beta_analytic
sigma_analytic = np.sqrt(np.mean(resid_analytic**2))

# Hessian (3x3) for (b1, b2, log_sigma):
#   Blocks:
#     H_bb = (1/sigma^2)*(X'X), 
#     H_b_logSigma = 0 (cross-partials vanish if modeling log_sigma),
#     H_logSigma_logSigma = n
# Explanation:
#   Negative log-likelihood for normal (in terms of log_sigma) is:
#     NLL = 0.5*n*log(2π) + n*log_sigma + 0.5*(1/σ^2)*Sum((y - Xb)^2),
#     with σ = exp(log_sigma).
#   => partial^2 wrt b,b is (X'X)/σ^2
#   => partial^2 wrt log_sigma, log_sigma is n
#   => cross partial b <-> log_sigma = 0 for an OLS structure.
# We evaluate at (beta_analytic, log_sigma_analytic).
#   log_sigma_analytic = ln(sigma_analytic)
H_bb = (X_np.T @ X_np) / (sigma_analytic**2)
n = X_np.shape[0]
H_logSigma_logSigma = n  # partial^2 wrt log_sigma
H_b_logSigma = np.zeros((2,1))  # cross-partials
H_analytic = np.block([
    [H_bb,        H_b_logSigma],
    [H_b_logSigma.T, H_logSigma_logSigma]
])

# -----------------------------------------------------------------
# 3. PYTORCH MLE
#    Parameterize as (b1, b2, log_sigma), minimize NLL
# -----------------------------------------------------------------
X_t = torch.tensor(X_np, dtype=torch.float32)
y_t = torch.tensor(y_np, dtype=torch.float32)
theta = torch.zeros(3, requires_grad=True)  # [b1, b2, log_sigma]

def neg_loglik(theta, X, y):
    b1, b2, log_sigma = theta
    sigma = torch.exp(log_sigma)
    y_hat = b1*X[:,0] + b2*X[:,1]
    resid = y - y_hat
    n = len(y)
    return 0.5*n*np.log(2*np.pi) + n*log_sigma + 0.5*torch.sum(resid**2)/(sigma**2)

optimizer = torch.optim.LBFGS([theta], lr=0.1)

def closure():
    optimizer.zero_grad()
    loss = neg_loglik(theta, X_t, y_t)
    loss.backward()
    return loss

for _ in range(50):
    optimizer.step(closure)

b1_torch, b2_torch, log_sigma_torch = theta.detach().numpy()
sigma_torch = np.exp(log_sigma_torch)

# Hessian from PyTorch (numerical)
def torch_hessian(fn, param):
    # param is a 1D torch tensor with requires_grad=True
    # returns Hessian as a NumPy array
    with torch.enable_grad():
        grad_1 = torch.autograd.grad(fn(param), param, create_graph=True)[0]
        dim = param.numel()
        H = torch.zeros(dim, dim)
        for i in range(dim):
            grad_2 = torch.autograd.grad(grad_1[i], param, retain_graph=True)
            H[i, :] = grad_2[0]
        return H.detach().numpy()

theta_t = theta.clone().requires_grad_(True)
H_torch = torch_hessian(lambda p: neg_loglik(p, X_t, y_t), theta_t)

# -----------------------------------------------------------------
# 4. STATSMODELS MLE (GenericLikelihoodModel)
# -----------------------------------------------------------------
class NormalMLE(GenericLikelihoodModel):
    def nloglikeobs(self, params):
        b1, b2, log_sigma = params
        sigma = np.exp(log_sigma)
        y_hat = b1*self.exog[:,0] + b2*self.exog[:,1]
        resid = self.endog - y_hat
        return 0.5*np.log(2*np.pi) + log_sigma + 0.5*(resid**2)/(sigma**2)

mle_mod = NormalMLE(endog=y_np, exog=X_np)
mle_res = mle_mod.fit(start_params=[0,0,0], method="bfgs", disp=False)
b1_sm, b2_sm, log_sigma_sm = mle_res.params
sigma_sm = np.exp(log_sigma_sm)

# Hessian from StatsModels is the second derivative of *log-likelihood*, so we get
# cov_params() from the *inverse* of the Hessian of the *negative* log-likelihood.
H_statsmodels = mle_res.cov_params()
# That is actually the covariance matrix. The Hessian can be approximated by inverse of that:
H_statsmodels_inv = np.linalg.inv(H_statsmodels)

# -----------------------------------------------------------------
# 5. TABLES
#   - First: parameter estimates from True, Analytical, PyTorch, StatsModels
#   - Second: Hessians from Analytical, PyTorch, StatsModels
# -----------------------------------------------------------------
# First table: parameter estimates
df_estimates = pd.DataFrame({
    "Param": ["b1", "b2", "sigma"],
    "True": [beta_true[0], beta_true[1], sigma_true],
    "Analytical MLE": [beta_analytic[0], beta_analytic[1], sigma_analytic],
    "PyTorch MLE": [b1_torch, b2_torch, sigma_torch],
    "StatsModels MLE": [b1_sm, b2_sm, sigma_sm]
})

# Second table: Hessians
# - We have 3x3 from each approach (b1, b2, log_sigma)
# - StatsModels gave us cov_params; invert to approximate Hessian
labels = ["b1", "b2", "log_sigma"]

# Flatten each Hessian in row-major order for quick comparison
df_hessian = pd.DataFrame({
    "Analytical": H_analytic.flatten(),
    "PyTorch": H_torch.flatten(),
    "StatsModels": H_statsmodels_inv.flatten()
}, index=[
    f"{i}_{j}" for i in labels for j in labels
])

# -----------------------------------------------------------------
# DISPLAY RESULTS
# -----------------------------------------------------------------
print("\n--- Parameter Estimates ---")
print(df_estimates.round(6))

print("\n--- Hessians (flattened 3x3) ---")
print(df_hessian.round(6))



--- Parameter Estimates ---
   Param  True  Analytical MLE  PyTorch MLE  StatsModels MLE
0     b1   1.0        0.978943     0.978944         0.978943
1     b2   2.0        2.042813     2.042810         2.042813
2  sigma   1.5        1.477383     1.477383         1.477382

--- Hessians (flattened 3x3) ---
                      Analytical      PyTorch  StatsModels
b1_b1                 148.902165   148.902161   148.902338
b1_b2                 113.320888   113.320892   113.321020
b1_log_sigma            0.000000     0.000400     0.000150
b2_b1                 113.320888   113.320892   113.321020
b2_b2                 158.375468   158.375473   158.375654
b2_log_sigma            0.000000     0.000776     0.000100
log_sigma_b1            0.000000     0.000406     0.000150
log_sigma_b2            0.000000     0.000776     0.000100
log_sigma_log_sigma  1000.000000  1999.999878  2000.002320


### PyTorch Autograd

- **Tensor Creation**: Define a scalar tensor $z$ with `requires_grad=True`.
- **Function Definition**: Let $y = z^2 + z$.  
  - Calling `y.backward()` computes $\frac{dy}{dz}$ automatically.  
  - The gradient is stored in `z.grad`.

- **Weighted Mean**: Let $\boldsymbol{x}$ be an $n$-dimensional vector, and $\boldsymbol{w}$ be weights.  
  - Compute $\mu = \frac{\boldsymbol{x} \boldsymbol{w}}{\sum \boldsymbol{w}}$.  
  - Using `autograd.grad(mu, w)` in PyTorch gives $\frac{\partial \mu}{\partial \boldsymbol{w}}$.  
  - Scaling by $n$ ensures consistency with the factor used in the original R code.

In [29]:
import torch
import numpy as np

# 1. Simple demonstration of tensors and autodiff
z = torch.tensor(0.5, requires_grad=True)  # analogous to .5 in R
y = z**2 + z
y.backward()  # differentiate y w.r.t. z
print("Value of z.grad:", z.grad.item())

# 2. Weighted Mean Example
torch.manual_seed(1234)
n = 100

# Generate data (like runif(n) in R)
x0 = np.random.rand(n)
x = torch.tensor(x0.reshape(1, -1), dtype=torch.float32)  # shape: (1, n)

# Create weights (all ones), then convert to torch
w0 = np.ones((n, 1), dtype=np.float32)
w = torch.tensor(w0, requires_grad=True)

# Weighted mean
mu = (x @ w) / torch.sum(w)

# Gradient of mu w.r.t. w
grad = torch.autograd.grad(mu, w)

# Multiply by n to mimic the R code's scaling
Lauto = n * grad[0].detach().numpy()

print("Weighted mean (mu):", mu.item())
print("Scaled gradient Lauto:", Lauto.flatten())


Value of z.grad: 2.0
Weighted mean (mu): 0.5020911693572998
Scaled gradient Lauto: [-0.36259884  0.38119125  0.14455152  0.28469217  0.29645973 -0.34025913
 -0.4394188   0.13489677 -0.16484235 -0.24163206 -0.3722259   0.35728598
  0.06934097  0.33461872  0.05465974 -0.45357817 -0.28257996  0.3151117
 -0.31083164  0.41022095 -0.1823383  -0.07971539  0.15996979  0.2979676
  0.26349908  0.3142938   0.00753673 -0.18019448 -0.2871688   0.457115
  0.07162821  0.46687686 -0.00871816 -0.12694216  0.10923948  0.49101588
  0.36262265 -0.09253472 -0.45909375 -0.0933989   0.25658783 -0.5003683
  0.33057648 -0.24175839 -0.13594106  0.41859215 -0.1279648  -0.02390249
 -0.05818801 -0.13917899 -0.4206222  -0.07822332 -0.0089433  -0.2837852
  0.32356018 -0.00751466 -0.26352262 -0.4017272   0.07573581 -0.02352423
  0.21313336  0.4862398  -0.3016582   0.23106983 -0.07058103 -0.10677893
 -0.21312293 -0.14453895 -0.38423768 -0.04669796  0.08548484 -0.22254871
 -0.04841904 -0.03005252  0.15750834  0.0814368